In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
# File path inside your Google Drive
input_path = "/content/drive/MyDrive/Colab Notebooks/sample_food_feedback.csv"
output_path = "/content/drive/MyDrive/Colab Notebooks/feedback_with_3_class_sentiment.csv"

In [1]:
!pip install transformers torch --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 69.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 52.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 31.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 79.3 MB/s eta 0:00:00


In [8]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import pandas as pd

model_name = "cardiffnlp/twitter-roberta-base-sentiment"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)
labels = ['NEGATIVE', 'NEUTRAL', 'POSITIVE']

In [9]:
def get_roberta_sentiment(text):
    if not text or not text.strip():
        return pd.Series(["NEUTRAL", 0.0])
    inputs = tokenizer(text, return_tensors="pt", truncation=True)
    outputs = model(**inputs)
    scores = torch.nn.functional.softmax(outputs.logits, dim=1)[0]
    label = labels[scores.argmax()]
    confidence = scores.max().item()
    return pd.Series([label, confidence])

In [10]:
# Load from Drive
df = pd.read_csv(input_path)
df = df.dropna()
df['feedback'] = df['feedback'].str.strip()

# Apply sentiment function
df[['sentiment', 'confidence']] = df['feedback'].apply(get_roberta_sentiment)

# Preview result
df.head()

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


,username,feedback,sentiment,confidence
0,user_1,The pasta was delicious and well seasoned.,POSITIVE,0.972387
1,user_2,I found the dessert too sweet for my taste.,NEGATIVE,0.779322
2,user_3,Loved the cozy ambiance and the service.,POSITIVE,0.984671
3,user_4,The food was okay but took too long to arrive.,NEUTRAL,0.512475
4,user_5,Amazing flavors! Will definitely come back.,POSITIVE,0.985340


In [11]:
df.to_csv(output_path, index=False)
print(f"Saved with sentiment: {output_path}")

Saved with sentiment: /content/drive/MyDrive/Colab Notebooks/feedback_with_3_class_sentiment.csv
